In [1]:
import ehrapy.api as ep

Unable to contact PyPI to check for the latest ehrapy version. Do you have an internet 
connection?

/home/zeth/miniconda3/envs/ehrapy/lib/python3.8/site-packages/medcat/cat.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Cleaning the input data

Just cleaning the input data based on https://colab.research.google.com/drive/1nQ3H7plYoOyC6MzqxECbm02oxoY6F3ZL#scrollTo=XHqZWgqoadQq

We will hide all of that for the final demo notebook and just read in the cleaned versions or something. The cleaning functions will be added to the ehrapy API at a later point.

# Reading in the cleaned data

Read in relevant files (notevents.csv and patients.csv) and create 1 AnnData object with both in one. If having both in 1 is not feasible we can go with 2 and possible a MuData object.

# Data overview

Read in vocabulary file with MedCAT and show one example.

Creating a vocabulary.

Read in concept database and show one example.

Create a concept database.

# Creating a model and predicting disease

https://colab.research.google.com/drive/1LL0EP_qN78ljs1PozaoiUDMaBo6zb4Gm

loading the vocabulary and concept DB and configuring stuff.

Show simple prediction on 1-2 examples.

Show details on the same prediction.

Mention that you can train your own model, but likely skip this here.

Run the prediction on the complete text column. Add columns to obs for relevant results. To make it easy put several results of the same type (e.g. if >1 diseases were found -> all of them into one cell in the column) into the same column.

Plot the distribution of e.g. disease type for an obs column as a bar plot. Maybe we can even filter the disease and show a nice UMAP? Should be easier of we filter for TUI before like the example does. Let's demonstrate this on a reasonable amount of data.

# Meta Annotations

https://colab.research.google.com/drive/1Cql-Hkt-OTK22FmY9tdfoVZSCbFIJAu4#scrollTo=f1yh0JBSUGJa

We should now demonstrate that we can not only predict disease, but also add the meta annotations.

Creating a tokenizer.

Train word2vec and assigning results to a variable. Don't think that they need to be part of the AnnData object.

Get embeddings for the text and saving them into the AnnData object somewhere.

https://colab.research.google.com/drive/1QpFq55xxzaZjAC5yWMpmWhWldfdkYBW9

I guess we skip supervised learning here as well.

Now we predict some meta annotations and add the results to our obs columns in a simplified manner as above.

Maybe some plots here again if needed.

# TESTING BY LUKAS

In [2]:
cat = ep.tl.MedCAT()

vocabulary = cat.load_vocabulary("vocab.dat")
concept_db = cat.load_concept_db("cdb-medmen-v1.dat")
cat.update_cat(vocabulary=vocabulary, concept_db=concept_db)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_53389/693909376.py:5 in <module>                                           │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_53389/693909376.py'                  │
│ /home/zeth/PycharmProjects/ehrapy/ehrapy/api/tools/nlp/_medcat.py:83 in update_cat        │
│                                                                                           │
│    80 │   │   return self.concept_db                                                      │
│    81 │                                                                                   │
│    82 │   def update_cat(self, vocabulary: Vocab = None, concept_db: CDB = None):         │
│ ❱  83 │   │   self.cat = CAT(cdb=concept_db, config=concept_db.config, vocab=vocabulary)  │
│    84 │                                                                                   │
│    85 │   def extract_entities_text(self, text: str) -> Doc:                              │
│    86 │   │   """Extracts entities for a provided text.                                   │
│                                                                                           │
│ /home/zeth/miniconda3/envs/ehrapy/lib/python3.8/site-packages/medcat/cat.py:79 in         │
│ __init__                                                                                  │
│                                                                                           │
│     76 │   │   self.log.setLevel(self.config.general['log_level'])                        │
│     77 │   │                                                                              │
│     78 │   │   # Build the pipeline                                                       │
│ ❱   79 │   │   self.pipe = Pipe(tokenizer=spacy_split_all, config=self.config)            │
│     80 │   │   self.pipe.add_tagger(tagger=tag_skip_and_punct,                            │
│     81 │   │   │   │   │   │   │    name='skip_and_punct',                                │
│     82 │   │   │   │   │   │   │    additional_fields=['is_punct'])                       │
│                                                                                           │
│ /home/zeth/miniconda3/envs/ehrapy/lib/python3.8/site-packages/medcat/pipe.py:39 in        │
│ __init__                                                                                  │
│                                                                                           │
│    36 │   # Add file and console handlers                                                 │
│    37 │   log = add_handlers(log)                                                         │
│    38 │   def __init__(self, tokenizer: Tokenizer, config: Config):                       │
│ ❱  39 │   │   self.nlp = spacy.load(config.general['spacy_model'],                        │
│       disable=config.general['spacy_disabled_components'])                                │
│    40 │   │   if config.preprocessing['stopwords'] is not None:                           │
│    41 │   │   │   self.nlp.Defaults.stop_words = set(config.preprocessing['stopwords'])   │
│    42 │   │   self.nlp.tokenizer = tokenizer(self.nlp, config)                            │
│                                                                                           │
│ /home/zeth/miniconda3/envs/ehrapy/lib/python3.8/site-packages/spacy/__init__.py:51 in     │
│ load                                                                                      │
│                                                                                           │
│   48 │   │   keyed by section values in dot notation.                                     │
│   49 │   RETURNS (Language): The loaded nlp object